# Предобработка твитов

In [ ]:
import pandas as pd
import torch
import transformers
import numpy as np
from tqdm import notebook
import ipywidgets as widgets


In [ ]:
# Загрузка предобученного токенизатора
tokenizer = transformers.BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

vector = tokenizer.encode('Очень удобно использовать уже готовый трансформатор текста', add_special_tokens=True)

n = 280
padded = np.array(vector + [0] * (n - len(vector)))

# создаём маску для важных токенов
attention_mask = np.where(padded != 0, 1, 0)

print(attention_mask.shape)




In [ ]:
# Загружаем предобученную модель
model = transformers.BertModel.from_pretrained("DeepPavlov/rubert-base-cased")

embeddings = []
batch_size = 100

# Проверяем, что размерность padded корректна
padded = np.expand_dims(padded, axis=0)
attention_mask = np.expand_dims(attention_mask, axis=0)


In [ ]:
if batch_size > 0:
    for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size * i:batch_size * (i + 1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size * i:batch_size * (i + 1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings.last_hidden_state[:, 0, :].numpy())

    # Объединяем все эмбеддинги
    features = np.concatenate(embeddings)
    print("Features shape:", features.shape)
else:
    print("Ошибка: недостаточно данных для хотя бы одного батча.")